# Predict the ratings of players
1. Player's rating on woogles.io BEFORE the game was played; ratings are per Lexicon / time control name (AKA game variant). In test.csv, ratings are NA for player games; this is what you are predicting.

2. The evaluation algorithm is RMSE

In [ ]:
import numpy as np
#%load_ext cudf.pandas
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# play link: https://playscrabble.com/play/ai

In [ ]:
games = pd.read_csv("/Users/sally/Documents/document/UMN_courses/MSBA6311_Predictive/team_project/games.csv")
sample_submission = pd.read_csv("/Users/sally/Documents/document/UMN_courses/MSBA6311_Predictive/team_project/sample_submission.csv")
test = pd.read_csv("/Users/sally/Documents/document/UMN_courses/MSBA6311_Predictive/team_project/test.csv")
train = pd.read_csv("/Users/sally/Documents/document/UMN_courses/MSBA6311_Predictive/team_project/train.csv")
turns = pd.read_csv("/Users/sally/Documents/document/UMN_courses/MSBA6311_Predictive/team_project/turns.csv")

# columns explanation for games.csv

game_id: 1~72773

first: 1314 unique player go first

time_control_name: ['regular', 'rapid', 'blitz', 'ultrablitz']

game_end_reason: ['STANDARD', 'RESIGNED', 'TIME', 'CONSECUTIVE_ZEROES']

winner: [1,  0, -1] I guess 1 mean the person go first win; 0 mean tie; -1 mean the person go second win

lexicon(字典): ['NWL20', 'CSW21', 'ECWL', 'NSWL20']

initial_time_seconds: Time limit each player has in the game (the game they plan to used)
[1200, 3600,  900, 1260, 1440,  720,  540,  300, 1320, 1080, 1800, 60, 1500,  780,  600, 1020, 1140,  420, 2400, 3300, 2700,  180, 840,  120, 2100,  480,  960,  360, 3000,  240,  660, 1380, 30,15]

increment_seconds: Time increment each player get reward each time they play a turn
[ 0,  5, 59,  1, 10,  7, 60,  3, 15, 30,  2]

rating mode: ['CASUAL', 'RATED']

max_overtime_minutes: How far past the initial time limit players can go before they timeout (the max overtime they can use during the game)
[ 1,  5, 10,  2,  0,  3,  4,  6]

game_duration_second: the time both players used?


In [ ]:
fgames.head()


,game_id,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
0,1,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,2,Super,regular,STANDARD,1,2022-08-10 19:19:59,CSW21,3600,0,RATED,1,364.214418
2,3,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
3,4,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141
4,5,STEEBot,regular,STANDARD,0,2022-09-06 04:31:36,NWL20,1200,0,CASUAL,1,642.688722


# columns in "games" to retain

"first": to know who win the game

"time_control_name" is quite same as "initial time second" so maybe choose one

game end reason: resigned may affect rating

different lexicon different rating process

casual like dummy

lose point on using max_overtime_minutes, compare max and initial

# Other idea for games.csv:
check correlation between game duration seconds and points(scores)

# Question left for games.csv
Q1 Does Casual mean doesn't count in rating?\
Q2 Not sure did I guess the meaning of winner right?

In [ ]:
turns.head()
# groupby game_id and nickname
# skip rack
# skip location

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
0,1,1,BetterBot,DDEGITT,8G,DIG,10,10,Play
1,1,2,stevy,AEHOPUX,7H,HAP,18,18,Play
2,1,3,BetterBot,DEELTTU,6I,LUTE,16,26,Play
3,1,4,stevy,EMORSUX,5K,UM,16,34,Play
4,1,5,BetterBot,ACDEITU,L5,..DICATE,28,54,Play


In [ ]:
turns[turns['game_id']==2]

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
27,2,1,Super,AFINRTV,8D,FAINT,24,24,Play
28,2,2,BetterBot,EEEEGIT,G6,GE.IE,8,8,Play
29,2,3,Super,AAERTVW,F10,WAVER,34,58,Play
30,2,4,BetterBot,EEHNNOT,14B,ENTH.ONE,72,80,Play
31,2,5,Super,AABEFTY,15G,FY,44,102,Play
32,2,6,BetterBot,DEOPPST,J8,STOPPED,85,165,Play
33,2,7,Super,AABCDET,K10,BEAD,33,135,Play
34,2,8,BetterBot,DEIJNNR,E4,NINJ.,24,189,Play
35,2,9,Super,ACEGIST,L6,CAGIEST,94,229,Play
36,2,10,BetterBot,ADEOOQR,8L,.OOD,18,207,Play


# columns in "turns" to retain
drop rack\
drop location

In [ ]:
turns['move'].unique()

array(['DIG', 'HAP', 'LUTE', ..., 'FERmENT', 'CAGOUL.', 'SPINTOS'],
      dtype=object)

In [ ]:
turns["turn_type"].unique()

array(['Play', 'End', 'Exchange', 'Pass', 'Timeout', 'Challenge',
       'Six-Zero Rule', nan], dtype=object)

In [ ]:
turns[turns['game_id'] == 12]

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
303,12,1,HastyBot,?ABMOSY,8G,BOY,16,16,Play
304,12,2,BB-8,ADEEEIK,9I,EKE,14,14,Play
305,12,3,HastyBot,?AILLMS,L9,SAwMILL,76,92,Play
306,12,4,BB-8,ACDEIIO,15L,.OCI,18,32,Play
307,12,5,HastyBot,EEGINOU,N9,EUGENI.,24,116,Play
308,12,6,BB-8,AADDEIN,7G,ADD,25,57,Play
309,12,7,HastyBot,EIIOOQV,NaN,-IOOQV,0,116,Exchange
310,12,8,BB-8,AEGINTT,G6,T..,5,62,Play
311,12,9,HastyBot,ACEEIIN,7F,C...IE,10,126,Play
312,12,10,BB-8,AAEGINT,K11,EAT,14,76,Play


In [ ]:
sum(turns["rack"].apply(lambda x : len(str(x)) <7))

# standard end run out of bags of words

206869

# some summary:

each player is suppose to have 1~9 kind of score, because there are three different dictionaries and 4 different initial time_control_name.\



# merging data set for training

In [ ]:
train_merge1 = train.set_index("game_id")
games_merge1 = games.set_index("game_id")
merge1 = train_merge1.join(games_merge1)
merge1.head()

,nickname,score,rating,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds
game_id,,,,,,,,,,,,,,
1,BetterBot,335,1637,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
1,stevy,429,1500,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274
3,davidavid,440,1811,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
3,BetterBot,318,2071,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262
4,Inandoutworker,119,1473,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141


In [ ]:
turns["turn_type"].unique()

array(['Play', 'End', 'Exchange', 'Pass', 'Timeout', 'Challenge',
       'Six-Zero Rule', nan], dtype=object)

# We can condider adding this or not

In [ ]:
# Group by 'game_id' and 'nickname', and then 'turn_type', and count the occurrences
turn_type_counts = turns.groupby(['game_id', 'nickname', 'turn_type']).size().reset_index(name='count')

# Pivot the table to get turn types as columns and their counts as values
pivot_table = turn_type_counts.pivot_table(index=['game_id', 'nickname'], columns='turn_type', values='count', fill_value=0)



# Reset the index to turn the MultiIndex into columns
pivot_table.reset_index(inplace=True)

# Flatten the columns after pivot
pivot_table.columns = [''.join(col).strip() if col[1] else col[0] for col in pivot_table.columns.values]

pivot_table



,game_id,nickname,Challenge,End,Exchange,Pass,Play,Six-Zero Rule,Timeout
0,1,BetterBot,0.0,0.0,0.0,0.0,13.0,0.0,0.0
1,1,stevy,0.0,1.0,0.0,0.0,13.0,0.0,0.0
2,2,BetterBot,0.0,0.0,0.0,0.0,12.0,0.0,0.0
3,2,Super,0.0,1.0,1.0,1.0,11.0,0.0,0.0
4,3,BetterBot,0.0,0.0,1.0,0.0,12.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
145541,72771,HastyBot,0.0,0.0,1.0,0.0,14.0,0.0,0.0
145542,72772,BetterBot,0.0,0.0,0.0,0.0,14.0,0.0,0.0
145543,72772,Gtowngrad,0.0,1.0,0.0,0.0,15.0,0.0,0.0
145544,72773,HastyBot,0.0,1.0,0.0,0.0,13.0,0.0,0.0


In [ ]:
turns[turns['game_id'] == 12]

# Last turn

,game_id,turn_number,nickname,rack,location,move,points,score,turn_type
303,12,1,HastyBot,?ABMOSY,8G,BOY,16,16,Play
304,12,2,BB-8,ADEEEIK,9I,EKE,14,14,Play
305,12,3,HastyBot,?AILLMS,L9,SAwMILL,76,92,Play
306,12,4,BB-8,ACDEIIO,15L,.OCI,18,32,Play
307,12,5,HastyBot,EEGINOU,N9,EUGENI.,24,116,Play
308,12,6,BB-8,AADDEIN,7G,ADD,25,57,Play
309,12,7,HastyBot,EIIOOQV,NaN,-IOOQV,0,116,Exchange
310,12,8,BB-8,AEGINTT,G6,T..,5,62,Play
311,12,9,HastyBot,ACEEIIN,7F,C...IE,10,126,Play
312,12,10,BB-8,AAEGINT,K11,EAT,14,76,Play


In [ ]:
mean_pts = turns.groupby(["game_id", "nickname"]).agg({'points': 'mean'})
mean_pts.rename(columns={"points":"mean_pts"},inplace=True)
mean_pts

mean_pts
game_id nickname            
1       BetterBot  25.769231
        stevy      30.642857
2       BetterBot  33.416667
        Super      34.857143
3       BetterBot  24.461538
...                      ...
72771   HastyBot   26.200000
72772   BetterBot  31.571429
        Gtowngrad  24.250000
72773   HastyBot   24.714286
        adola      31.916667

[145546 rows x 1 columns]

In [ ]:
last_score = turns.groupby(["game_id", "nickname"]).agg({'score': 'last'})
last_score.rename(columns={"score":"last_score"},inplace=True)
last_score

last_score
game_id nickname             
1       BetterBot         335
        stevy             429
2       BetterBot         401
        Super             488
3       BetterBot         318
...                       ...
72771   HastyBot          393
72772   BetterBot         442
        Gtowngrad         388
72773   HastyBot          346
        adola             383

[145546 rows x 1 columns]

In [ ]:
count_turn = turns.groupby(["game_id", "nickname"]).agg({"turn_number": "count"})
count_turn.rename(columns={"turn_number":"count_turn"},inplace=True)
count_turn

count_turn
game_id nickname             
1       BetterBot          13
        stevy              14
2       BetterBot          12
        Super              14
3       BetterBot          13
...                       ...
72771   HastyBot           15
72772   BetterBot          14
        Gtowngrad          16
72773   HastyBot           14
        adola              12

[145546 rows x 1 columns]

In [ ]:
merge1_trans = merge1.reset_index()
merge1_trans = merge1_trans.set_index(["game_id","nickname"])
merge2 = merge1_trans.join(mean_pts)
merge3 = merge2.join(last_score)
merge4 = merge3.join(count_turn)
merge4 = merge4.reset_index()
merge4.head()

,game_id,nickname,score,rating,first,time_control_name,game_end_reason,winner,created_at,lexicon,initial_time_seconds,increment_seconds,rating_mode,max_overtime_minutes,game_duration_seconds,mean_pts,last_score,count_turn
0,1,BetterBot,335,1637,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,25.769231,335,13
1,1,stevy,429,1500,BetterBot,regular,STANDARD,1,2022-08-26 03:38:49,NWL20,1200,0,CASUAL,1,674.844274,30.642857,429,14
2,3,davidavid,440,1811,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,31.428571,440,14
3,3,BetterBot,318,2071,BetterBot,regular,STANDARD,1,2022-09-04 08:04:27,CSW21,900,0,RATED,5,492.268262,24.461538,318,13
4,4,Inandoutworker,119,1473,BetterBot,regular,RESIGNED,0,2022-09-12 02:36:19,CSW21,3600,0,CASUAL,1,350.861141,8.500000,119,14


In [ ]:
# get dummy
categorical_columns = ['game_end_reason', 'winner', 'lexicon', 'rating_mode']

# Perform One-Hot Encoding
merge4 = pd.get_dummies(merge4, columns=categorical_columns)

merge5 = merge4.drop(["game_id","nickname","first", "time_control_name","created_at"],axis=1) #first 先drop掉

In [ ]:
has_null = merge5.isnull().values.any()
has_null_columns = merge5.isnull().any()
has_null_rows = merge5.isnull().any(axis=1)
print(has_null)
print()
print(has_null_columns)

False

score                                 False
rating                                False
initial_time_seconds                  False
increment_seconds                     False
max_overtime_minutes                  False
game_duration_seconds                 False
mean_pts                              False
last_score                            False
count_turn                            False
game_end_reason_CONSECUTIVE_ZEROES    False
game_end_reason_RESIGNED              False
game_end_reason_STANDARD              False
game_end_reason_TIME                  False
winner_-1                             False
winner_0                              False
winner_1                              False
lexicon_CSW21                         False
lexicon_ECWL                          False
lexicon_NSWL20                        False
lexicon_NWL20                         False
rating_mode_CASUAL                    False
rating_mode_RATED                     False
dtype: bool


In [ ]:
# normalization & train test split
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

X = merge5.drop(['rating'], axis=1)
y = merge5['rating']

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
import numpy as np

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor

In [ ]:


NUM_TRIALS = 1

p_grid_lr = {}  # Linear regression usually doesn't need hyperparameter tuning for basic use

p_grid_knn = {
    'n_neighbors': [3, 5, 7, 10],
    'weights': ['uniform', 'distance']
}

p_grid_tree = {
    'max_depth': [1, 3, 5, 7],
    'min_samples_split': [2, 3, 4]
}

p_grid_svr = {
    'kernel': ['rbf','linear'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto'],
    'epsilon': [0.01, 0.1, 0.5]
}

p_grid_rf = {
    'n_estimators': [10, 50, 100],
    'max_features': ['auto', 'sqrt', 'log2']
}

#p_grid_gb = {
#    'n_estimators': [100, 200],
#    'learning_rate': [0.01, 0.1, 0.5],
#    'max_depth': [3, 5, 7]
#}

p_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9]
}

def rmse(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    return np.sqrt(mse)

# Define a scorer that uses the RMSE function
rmse_scorer = make_scorer(rmse, greater_is_better=False)


def evaluate_model(model, param_grid, X, y, num_trials=NUM_TRIALS):
    non_nested_scores = np.zeros(num_trials)
    nested_scores = np.zeros(num_trials)
    best_params = []

    for i in range(num_trials):

        inner_cv = KFold(n_splits=4, shuffle=True, random_state=i) #random_state = i, This ensures that each trial of the cross-validation shuffles the data in a unique way, allowing for a more robust estimate of the model's performance.
        outer_cv = KFold(n_splits=4, shuffle=True, random_state=i)

        # GridSearchCV automatically refits the model with the best set of hyperparameters using the whole dataset
        clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, scoring=rmse_scorer)
        clf.fit(X, y)

        non_nested_scores[i] = -clf.best_score_  # Negate the score to get the positive RMSE
        best_params.append(clf.best_params_)

        # Nested CV with parameter optimization
        nested_score = cross_val_score(clf, X=X, y=y, cv=outer_cv, scoring=rmse_scorer)
        nested_scores[i] = -nested_score.mean()

    # Average the scores across the different trials
    avg_non_nested_score = non_nested_scores.mean()
    avg_nested_score = nested_scores.mean()

    # Standard deviations of the scores
    std_non_nested_score = non_nested_scores.std()
    std_nested_score = nested_scores.std()

    return avg_non_nested_score, std_non_nested_score, avg_nested_score, std_nested_score, best_params

models = [
    (LinearRegression(), p_grid_lr),
    (KNeighborsRegressor(), p_grid_knn),
    (DecisionTreeRegressor(), p_grid_tree),
    (SVR(), p_grid_svr),
    (RandomForestRegressor(), p_grid_rf),
    #(GradientBoostingRegressor(), p_grid_gb),
    (XGBRegressor(), p_grid_xgb)
]

for model, params in models:
    non_nested_avg_score, non_nested_std_score, nested_avg_score, nested_std_score, best_params = evaluate_model(model, params, X_normalized, y)
    print(f"{model.__class__.__name__}: Non-Nested CV Avg RMSE: {non_nested_avg_score:.3f}, Std Dev: {non_nested_std_score:.3f}")
    print(f"{model.__class__.__name__}: Nested CV Avg RMSE: {nested_avg_score:.3f}, Std Dev: {nested_std_score:.3f}")
    print(f"Best parameters found: {best_params}\n")


LinearRegression: Non-Nested CV Avg RMSE: 142.063, Std Dev: 0.009
LinearRegression: Nested CV Avg RMSE: 142.063, Std Dev: 0.009
Best parameters found: [{}, {}, {}, {}, {}]

KNeighborsRegressor: Non-Nested CV Avg RMSE: 122.354, Std Dev: 0.133
KNeighborsRegressor: Nested CV Avg RMSE: 122.354, Std Dev: 0.133
Best parameters found: [{'n_neighbors': 10, 'weights': 'uniform'}, {'n_neighbors': 10, 'weights': 'uniform'}, {'n_neighbors': 10, 'weights': 'uniform'}, {'n_neighbors': 10, 'weights': 'uniform'}, {'n_neighbors': 10, 'weights': 'uniform'}]

DecisionTreeRegressor: Non-Nested CV Avg RMSE: 128.154, Std Dev: 0.177
DecisionTreeRegressor: Nested CV Avg RMSE: 128.158, Std Dev: 0.178
Best parameters found: [{'max_depth': 7, 'min_samples_split': 2}, {'max_depth': 7, 'min_samples_split': 2}, {'max_depth': 7, 'min_samples_split': 4}, {'max_depth': 7, 'min_samples_split': 3}, {'max_depth': 7, 'min_samples_split': 3}]



# merging data set for testing

In [ ]:
test_merge1 = test.set_index("game_id")
games_merge1 = games.set_index("game_id")
merge1 = test_merge1.join(games_merge1)
merge1.head()

merge1_trans = merge1.reset_index()
merge1_trans = merge1_trans.set_index(["game_id","nickname"])
merge2 = merge1_trans.join(mean_pts)
merge3 = merge2.join(last_score)
merge4 = merge3.join(count_turn)
merge4 = merge4.reset_index()
merge4.head()

# get dummy
categorical_columns = ['game_end_reason', 'winner', 'lexicon', 'rating_mode']

# Perform One-Hot Encoding
merge4 = pd.get_dummies(merge4, columns=categorical_columns)

merge5 = merge4.drop(["game_id","nickname","first", "time_control_name","created_at"],axis=1) #first 先drop掉

merge5.head()

test_X = merge5.drop(['rating'], axis=1)
# y = merge5['rating']



has_null = test_X.isnull().values.any()
#has_null_columns = test_X.isnull().any()
#has_null_rows = test_X_normalized.isnull().any(axis=1)
print(has_null)
#print()
#print(has_null_columns)

False

score                                 False
initial_time_seconds                  False
increment_seconds                     False
max_overtime_minutes                  False
game_duration_seconds                 False
mean_pts                              False
last_score                            False
count_turn                            False
game_end_reason_CONSECUTIVE_ZEROES    False
game_end_reason_RESIGNED              False
game_end_reason_STANDARD              False
game_end_reason_TIME                  False
winner_-1                             False
winner_0                              False
winner_1                              False
lexicon_CSW21                         False
lexicon_ECWL                          False
lexicon_NWL20                         False
rating_mode_CASUAL                    False
rating_mode_RATED                     False
dtype: bool


In [ ]:
X.shape

(100820, 21)

In [ ]:
test_X.shape

(44726, 20)

In [ ]:
test1 = X
test2 = test_X


In [ ]:
test2.insert(test1.columns.get_loc('lexicon_NSWL20'), 'lexicon_NSWL20', False)
test2.shape

(44726, 21)

In [ ]:
test_X.shape

(44726, 21)

In [ ]:
scaler = StandardScaler()
test_X_normalized = scaler.fit_transform(test_X)

In [ ]:
knn_model = KNeighborsRegressor(n_neighbors=10, weights='uniform')

knn_model.fit(X_normalized, y)

predictions = knn_model.predict(test_X_normalized)

In [ ]:
tree_model = DecisionTreeRegressor(max_depth = 7, min_samples_split= 2)
tree_model.fit(X_normalized, y)
predictions = tree_model.predict(test_X_normalized)

In [ ]:
test = pd.read_csv("/Users/chenshaokai/Desktop/UMN/semester2/predictive analytics/team project/scrabble-player-rating/test.csv")
test['rating'] = predictions
test = test[~test['nickname'].str.endswith('Bot')]

In [ ]:
submission = test[['game_id','rating']]
submission

,game_id,rating
0,2,2070.324425
3,7,2044.426249
5,11,1682.177062
7,14,1934.741246
8,27,2070.324425
...,...,...
44717,72760,2070.324425
44718,72761,1768.203364
44721,72762,1762.811828
44723,72768,2004.553632


In [ ]:
submission.to_csv("/Users/sally/Documents/document/UMN_courses/MSBA6311_Predictive/team_project/submission_tree.csv", index=False)

In [ ]:
origin_test = pd.read_csv("/Users/sally/Documents/document/UMN_courses/MSBA6311_Predictive/team_project/test.csv")
origin_test

,game_id,nickname,score,rating
0,2,Super,488,NaN
1,2,BetterBot,401,2000.0
2,7,STEEBot,377,2082.0
3,7,Priya1,379,NaN
4,11,STEEBot,334,1829.0
...,...,...,...,...
44721,72762,kyjo55555,367,NaN
44722,72768,HastyBot,524,2356.0
44723,72768,Maximilian,357,NaN
44724,72769,STEEBot,626,2110.0


In [ ]:
sample_submission

,game_id,rating
0,2,1790.322674
1,7,1790.322674
2,11,1790.322674
3,14,1790.322674
4,27,1790.322674
...,...,...
22358,72760,1790.322674
22359,72761,1790.322674
22360,72762,1790.322674
22361,72768,1790.322674
